Cruce de predicción del municipio vs chinchetas

In [3]:
#Librerías
import pandas as pd
import geopandas as gpd
import sys
sys.path.append(r"D:\Repositorios\geoloc2\preparacion_datos")
from funciones_cruces_limpieza import *
sys.path.append(r"D:\Repositorios\geoloc2\Valor_Catastral")
from Herramientas import *
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'funciones_cruces_limpieza'

In [5]:
def choose_city(gdf_zonas):
    '''
        Función para localizar el municipio a tratar dentro del shape de zonas rurales y urbanas
    '''
    zona = input('Digita el código de la zona que quieres (Ej. Ixtapan = 040):')
    gdf_mun = gdf_zonas[gdf_zonas.CVE_MUN == zona]
    #gdf_mun.plot()
    return gdf_mun
    
def cruce_limpieza_shapes(gdf_zonas,gdf_construcciones):
    gdf_mun = choose_city(gdf_zonas)
    gdf_mun = gdf_mun.to_crs(3857)
    gdf_cruce= gdf_construcciones.sjoin(gdf_mun,how='left')
    
    return gdf_cruce

Cruce de chinchetas vs denue

In [ ]:
pines = gpd.read_file(r)

In [ ]:
#La columna de clasificacion de chinchetas debe llamarse Clase, sino cambiar
pines.rename(columns={'clase_chin':'Clase'},inplace=True)

In [ ]:
denue= gpd.read_file(r)

In [ ]:
schema_crs={'set_ch':3857,'to_ch':3857,'set_dnue':6364,'to_dnue':3857}
pines = pines.set_crs(schema_crs['set_ch'],allow_override=True)
denue   = denue.set_crs(schema_crs['set_dnue'],allow_override=True)
pines.set_geometry('geometry',inplace=True)

In [ ]:
cruce_pinesxdenue=cruzar_chinchetas_vs_dnue(pines,denue)
#Buscar columna de denue de clasificacion 
cruce_pinesxdenue['Clase']

In [2]:
import numpy as np 
x='hh|jj|hh|jj|ll|i|o'
('|'.join(np.unique(np.array(x.split('|')))))

hh|i|jj|ll|o


In [ ]:
cruce_pinesxdenue.to_file(r"/Atlacomulco_casas_pinesxdenue.shp")

Cruce de casas x anterior 

In [7]:
detect_mun_casas = gpd.read_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/geoshaps/lerma/30mayo23/Lerma_casas.shp')
detect_mun_casas.tail(2)

,clase_dete,conf,area,geometry
0,establecimiento,0.40967,786.788071,"POLYGON ((-11084285.467 2201378.952, -11084275..."
1,establecimiento,0.28540,5887.411719,"POLYGON ((-11084248.799 2201298.712, -11084167..."
2,establecimiento,0.36865,593.947858,"POLYGON ((-11084204.616 2201225.041, -11084186..."
3,establecimiento,0.38403,2627.769310,"POLYGON ((-11084265.714 2201234.103, -11084219..."
4,casas,0.54443,152.986569,"POLYGON ((-11084220.602 2201199.380, -11084211..."
...,...,...,...,...
140841,establecimiento,0.45068,116.989730,"POLYGON ((-11063324.133 2191249.844, -11063314..."
140842,establecimiento,0.54395,229.479854,"POLYGON ((-11063325.338 2191262.917, -11063310..."
140843,establecimiento,0.68652,150.415367,"POLYGON ((-11063316.013 2191281.114, -11063303..."
140844,establecimiento,0.84375,246.835473,"POLYGON ((-11063302.961 2191266.723, -11063285..."


In [ ]:
cruce_casasxpines=gpd.sjoin(detect_mun_casas,cruce_pinesxdenue, how='left')
cruce_casasxpines

In [ ]:
#Filtrar las detecciones de casas que cruzan con chinchetas, buacar la columna correspondiente a la deteccion de casas de alexnet de la base de deteccion
cruce_casasxpines.loc[cruce_casasxpines.Clase.notna(),"clase_dete"] = 'establecimiento'

In [ ]:
#Filtrar las detecciones de casas que cruzan con chinchetas, buacar la columna correspondiente a la deteccion de casas de alexnet de la base de deteccion
cruce_casasxpines.loc[(cruce_casasxpines.clase_dete== 'establecimiento') & (cruce_casasxpines.Clase.isna())] = 'casas'

In [27]:
print('Columas de casas: ', detect_mun_casas.columns)
if input('Quiere crear la columna ID_casas') in ['1','si']:
    detect_mun_casas.reset_index(drop=False, inplace=True)
    detect_mun_casas.rename(columns={'index':'ID_casas'},inplace=True)

Columas de casas:  Index(['index', 'clase_dete', 'conf', 'area', 'geometry'], dtype='object')
Quiere crear la columna ID_casas1


La siguiente celda es para saber cuántos valores únicos hay en cada base

In [28]:
len(detect_mun_casas.geometry.unique())#,len(pines.geometry.unique())

140846

Cruce para identificación de zonas rurales y urbanas

In [29]:
#Lectura de base de zonas
zonas = gpd.read_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/bases_shp_INEGI/zonas_rurales_urbanas/15m.shp')
zonas.tail(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,AMBITO,TIPOMZA,geometry
160098,1512500010016004,15,125,0001,0016,004,Urbana,Típica,"POLYGON ((2807844.209 857424.049, 2807852.408 ..."
160099,1512500010016025,15,125,0001,0016,025,Urbana,Típica,"POLYGON ((2807780.538 857336.969, 2807799.790 ..."


Cruce de zonas vs construcciones

In [30]:
#Base de Areas verdes, camellones, glorietas
sia = gpd.read_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/bases_shp_INEGI/bases_areasv_camellones/15sia.shp')
sia.tail(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CONDICION,GEOGRAFICO,NOMSERV,TIPO,CVE_SERV,AMBITO,geometry
15162,151250007,15,125,0007,No Aplica,Camellón,Ninguno,Camellón,8,Urbana,"POLYGON ((2806139.047 856295.964, 2806184.291 ..."
15163,151250007,15,125,0007,No Aplica,Camellón,Ninguno,Camellón,1,Urbana,"POLYGON ((2806526.329 856391.304, 2806544.956 ..."


In [31]:
#Base de Palacios municipales, parques y jardines
sip=gpd.read_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/bases_shp_INEGI/bases_areasv_camellones/15sip.shp')
sip.tail(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,CONDICION,GEOGRAFICO,NOMSERV,TIPO,CVE_SERV,AMBITO,geometry
27662,1512500070054033,15,125,0007,0054,033,En Operación,Instalación de Servicios,Ayuntamiento Tonatitla,Ayudantía,2,Urbana,POINT (2806166.820 856249.282)
27663,1512500070054033,15,125,0007,0054,033,En Operación,Instalación Deportiva o Recreativa,Ninguno,Parque,1,Urbana,POINT (2806198.570 856264.945)


In [32]:
#Areas sin concentración Urbana
base_pe=gpd.read_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/bases_shp_INEGI/bases_areasv_camellones/15pe.shp')
base_pe.tail(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,geometry
3734,151250004,15,125,0004,"POLYGON ((2807618.090 858003.488, 2807623.898 ..."
3735,151250003,15,125,0003,"POLYGON ((2807943.254 855547.171, 2807979.521 ..."


In [33]:
#Base del cruce con areas rurales y urbanas   casas vs 15m
#base_ur = gpd.read_file(r"D:\Secretaría\cruces_bases\Zona_rurales_urbanas\crucexzonas_rurales_urbanas.shp")
base_ur = gpd.sjoin(detect_mun_casas,zonas.to_crs(3857),how='left')
base_ur.rename(columns={'index_right':'ID_15m'},inplace=True)
base_ur.shape, detect_mun_casas.shape

((143079, 14), (140846, 5))

In [34]:
len(base_ur.geometry.unique())

140846

In [35]:
base_ur[base_ur.geometry.isin(base_ur.geometry.value_counts()[base_ur.geometry.value_counts()>1].index)].tail(2)

,ID_casas,clase_dete,conf,area,geometry,ID_15m,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,AMBITO,TIPOMZA
140756,140756,establecimiento,0.49634,163.914182,"POLYGON ((-11063510.168 2191134.690, -11063500...",65094.0,1505100150167022,15,051,0015,0167,022,Rural,Típica
140756,140756,establecimiento,0.49634,163.914182,"POLYGON ((-11063510.168 2191134.690, -11063500...",63784.0,1505100150167018,15,051,0015,0167,018,Rural,Típica


In [36]:
cruce_sip=cruce_limpieza_shapes(sip,base_ur)
cruce_sip

Digita el código de la zona que quieres (Ej. Ixtapan = 040):051


,ID_casas,clase_dete,conf,area,geometry,ID_15m,CVEGEO_left,CVE_ENT_left,CVE_MUN_left,CVE_LOC_left,...,CVE_MUN_right,CVE_LOC_right,CVE_AGEB_right,CVE_MZA_right,CONDICION,GEOGRAFICO,NOMSERV,TIPO,CVE_SERV,AMBITO_right
0,0,establecimiento,0.40967,786.788071,"POLYGON ((-11084285.467 2201378.952, -11084275...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,establecimiento,0.28540,5887.411719,"POLYGON ((-11084248.799 2201298.712, -11084167...",131669.0,1510602383443001,15,106,0238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,establecimiento,0.36865,593.947858,"POLYGON ((-11084204.616 2201225.041, -11084186...",131669.0,1510602383443001,15,106,0238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,establecimiento,0.38403,2627.769310,"POLYGON ((-11084265.714 2201234.103, -11084219...",131669.0,1510602383443001,15,106,0238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,casas,0.54443,152.986569,"POLYGON ((-11084220.602 2201199.380, -11084211...",131669.0,1510602383443001,15,106,0238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140841,140841,establecimiento,0.45068,116.989730,"POLYGON ((-11063324.133 2191249.844, -11063314...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140842,140842,establecimiento,0.54395,229.479854,"POLYGON ((-11063325.338 2191262.917, -11063310...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140843,140843,establecimiento,0.68652,150.415367,"POLYGON ((-11063316.013 2191281.114, -11063303...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140844,140844,establecimiento,0.84375,246.835473,"POLYGON ((-11063302.961 2191266.723, -11063285...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#Combinar las columnas que se duplican porque están en ambas bases y crear una general
cruce_sip['CVEGEO']=cruce_sip.CVEGEO_left.combine_first(cruce_sip.CVEGEO_right)
cruce_sip['CVE_ENT']=cruce_sip.CVE_ENT_left.combine_first(cruce_sip.CVE_ENT_right)
cruce_sip['CVE_MUN']=cruce_sip.CVE_MUN_left.combine_first(cruce_sip.CVE_MUN_right)
cruce_sip['CVE_LOC']=cruce_sip.CVE_LOC_left.combine_first(cruce_sip.CVE_LOC_right)
cruce_sip['CVE_AGEB']=cruce_sip.CVE_AGEB_left.combine_first(cruce_sip.CVE_AGEB_right)
cruce_sip['CVE_MZA']=cruce_sip.CVE_MZA_left.combine_first(cruce_sip.CVE_MZA_right)
cruce_sip['AMBITO']=cruce_sip.AMBITO_left.combine_first(cruce_sip.AMBITO_right)
cruce_sip.rename(columns={'index_right':'ID_15sip'},inplace=True)

In [38]:
#Eliminar las columnas divididas porque ya están en la general
cruce_sip.drop(['CVEGEO_left','CVEGEO_right'],axis=1,inplace=True)
cruce_sip.drop(['CVE_ENT_left','CVE_ENT_right'],axis=1,inplace=True)
cruce_sip.drop(['CVE_MUN_left','CVE_MUN_right'],axis=1,inplace=True)
cruce_sip.drop(['CVE_LOC_left','CVE_LOC_right'],axis=1,inplace=True)
cruce_sip.drop(['CVE_AGEB_left','CVE_AGEB_right'],axis=1,inplace=True)
cruce_sip.drop(['CVE_MZA_left','CVE_MZA_right'],axis=1,inplace=True)
cruce_sip.drop(['AMBITO_left','AMBITO_right'],axis=1,inplace=True)


In [39]:
cruce_sip.columns

Index(['ID_casas', 'clase_dete', 'conf', 'area', 'geometry', 'ID_15m',
       'TIPOMZA', 'ID_15sip', 'CONDICION', 'GEOGRAFICO', 'NOMSERV', 'TIPO',
       'CVE_SERV', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_AGEB',
       'CVE_MZA', 'AMBITO'],
      dtype='object')

In [40]:
cruce_sia = cruce_limpieza_shapes(sia,cruce_sip)
cruce_sia.tail(2)

Digita el código de la zona que quieres (Ej. Ixtapan = 040):051


,ID_casas,clase_dete,conf,area,geometry,ID_15m,TIPOMZA,ID_15sip,CONDICION_left,GEOGRAFICO_left,...,CVEGEO_right,CVE_ENT_right,CVE_MUN_right,CVE_LOC_right,CONDICION_right,GEOGRAFICO_right,NOMSERV_right,TIPO_right,CVE_SERV_right,AMBITO_right
140844,140844,establecimiento,0.84375,246.835473,"POLYGON ((-11063302.961 2191266.723, -11063285...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140845,140845,establecimiento,0.57666,77.136085,"POLYGON ((-11063325.336 2191099.119, -11063317...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#Combinar las columnas que se duplican porque están en ambas bases y crear una general
cruce_sia['CONDICION']=cruce_sia.CONDICION_left.combine_first(cruce_sia.CONDICION_right)
cruce_sia['GEOGRAFICO']=cruce_sia.GEOGRAFICO_left.combine_first(cruce_sia.GEOGRAFICO_right)
cruce_sia['NOMSERV']=cruce_sia.NOMSERV_left.combine_first(cruce_sia.NOMSERV_right)
cruce_sia['TIPO']=cruce_sia.TIPO_left.combine_first(cruce_sia.TIPO_right)
cruce_sia['CVE_SERV']=cruce_sia.CVE_SERV_left.combine_first(cruce_sia.CVE_SERV_right)
cruce_sia['CVEGEO']=cruce_sia.CVEGEO_left.combine_first(cruce_sia.CVEGEO_right)
cruce_sia['CVE_ENT']=cruce_sia.CVE_ENT_left.combine_first(cruce_sia.CVE_ENT_right)
cruce_sia['CVE_MUN']=cruce_sia.CVE_MUN_left.combine_first(cruce_sia.CVE_MUN_right)
cruce_sia['CVE_LOC']=cruce_sia.CVE_LOC_left.combine_first(cruce_sia.CVE_LOC_right)
cruce_sia['AMBITO']=cruce_sia.AMBITO_left.combine_first(cruce_sia.AMBITO_right)
cruce_sia.rename(columns={'index_right':'ID_15sia'},inplace=True)

In [42]:
#Eliminar las columnas divididas porque ya están en la general
cruce_sia.drop(['CONDICION_left','CONDICION_right'],axis=1,inplace=True)
cruce_sia.drop(['GEOGRAFICO_left','GEOGRAFICO_right'],axis=1,inplace=True)
cruce_sia.drop(['NOMSERV_left','NOMSERV_right'],axis=1,inplace=True)
cruce_sia.drop(['TIPO_left','TIPO_right'],axis=1,inplace=True)
cruce_sia.drop(['CVE_SERV_left','CVE_SERV_right'],axis=1,inplace=True)
cruce_sia.drop(['CVEGEO_left','CVEGEO_right'],axis=1,inplace=True)
cruce_sia.drop(['CVE_ENT_left','CVE_ENT_right'],axis=1,inplace=True)
cruce_sia.drop(['CVE_MUN_left','CVE_MUN_right'],axis=1,inplace=True)
cruce_sia.drop(['CVE_LOC_left','CVE_LOC_right'],axis=1,inplace=True)
cruce_sia.drop(['AMBITO_left','AMBITO_right'],axis=1,inplace=True)


In [43]:
cruce_pe = cruce_limpieza_shapes(base_pe,cruce_sia)
cruce_pe.tail(2)

Digita el código de la zona que quieres (Ej. Ixtapan = 040):051


,ID_casas,clase_dete,conf,area,geometry,ID_15m,TIPOMZA,ID_15sip,CVE_AGEB,CVE_MZA,...,CVEGEO_left,CVE_ENT_left,CVE_MUN_left,CVE_LOC_left,AMBITO,index_right,CVEGEO_right,CVE_ENT_right,CVE_MUN_right,CVE_LOC_right
140844,140844,establecimiento,0.84375,246.835473,"POLYGON ((-11063302.961 2191266.723, -11063285...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1312.0,150510015,15,051,0015
140845,140845,establecimiento,0.57666,77.136085,"POLYGON ((-11063325.336 2191099.119, -11063317...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1312.0,150510015,15,051,0015


In [44]:
#Combinar las columnas que se duplican porque están en ambas bases y crear una general

cruce_pe['CVEGEO']=cruce_pe.CVEGEO_left.combine_first(cruce_pe.CVEGEO_right)
cruce_pe['CVE_ENT']=cruce_pe.CVE_ENT_left.combine_first(cruce_pe.CVE_ENT_right)
cruce_pe['CVE_MUN']=cruce_pe.CVE_MUN_left.combine_first(cruce_pe.CVE_MUN_right)
cruce_pe['CVE_LOC']=cruce_pe.CVE_LOC_left.combine_first(cruce_pe.CVE_LOC_right)


#Eliminar las columnas divididas porque ya están en la general

cruce_pe.drop(['CVEGEO_left','CVEGEO_right'],axis=1,inplace=True)
cruce_pe.drop(['CVE_ENT_left','CVE_ENT_right'],axis=1,inplace=True)
cruce_pe.drop(['CVE_MUN_left','CVE_MUN_right'],axis=1,inplace=True)
cruce_pe.drop(['CVE_LOC_left','CVE_LOC_right'],axis=1,inplace=True)
cruce_pe.rename(columns={'index_right':'ID_15pe'},inplace=True)

In [45]:
cruce_pe.columns

Index(['ID_casas', 'clase_dete', 'conf', 'area', 'geometry', 'ID_15m',
       'TIPOMZA', 'ID_15sip', 'CVE_AGEB', 'CVE_MZA', 'ID_15sia', 'CONDICION',
       'GEOGRAFICO', 'NOMSERV', 'TIPO', 'CVE_SERV', 'AMBITO', 'ID_15pe',
       'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC'],
      dtype='object')

In [46]:
try :
    cruce_pe = cruce_pe[['ID_casas', 'clase_dete', 'conf', 'area', 'Z', 'geometry', 'TIPOMZA', 'TIPO',
                     'ID_15m','ID_15sip', 'ID_15sia', 'ID_15pe',
                     'CVE_AGEB', 'CVE_MZA', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC','CVE_SERV',
                     'GEOGRAFICO', 'NOMSERV',  'AMBITO', 'CONDICION']]
except:
    cruce_pe = cruce_pe[['ID_casas', 'clase_dete', 'conf', 'area', 'geometry', 'TIPOMZA', 'TIPO',
                     'ID_15m','ID_15sip', 'ID_15sia', 'ID_15pe',
                     'CVE_AGEB', 'CVE_MZA', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC','CVE_SERV',
                     'GEOGRAFICO', 'NOMSERV',  'AMBITO', 'CONDICION']]

In [47]:
cruce_pe.columns

Index(['ID_casas', 'clase_dete', 'conf', 'area', 'geometry', 'TIPOMZA', 'TIPO',
       'ID_15m', 'ID_15sip', 'ID_15sia', 'ID_15pe', 'CVE_AGEB', 'CVE_MZA',
       'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_SERV', 'GEOGRAFICO',
       'NOMSERV', 'AMBITO', 'CONDICION'],
      dtype='object')

In [48]:
cruce_pe['CONDICION'].value_counts()

En Operación    508
No Aplica       495
Name: CONDICION, dtype: int64

In [57]:
cruce_pe['AMBITO'].value_counts()

Urbana    40889
Rural     23719
Name: AMBITO, dtype: int64

In [56]:
cruce_pe['NOMSERV'].value_counts()

Ninguno                               410
Universidad Autónoma Metropolitana     64
Angela Peralta                          1
Heroes de Chapultepec                   1
Maria Montessori                        1
Prof. Esperanza Gómez de Vega           1
Salazar                                 1
Name: NOMSERV, dtype: int64

In [55]:
cruce_pe['GEOGRAFICO'].value_counts()

Escuela                               251
Camellón                               70
Templo                                 45
Cementerio                             42
Centro de Asistencia Médica            20
Instalación Deportiva o Recreativa     19
Instalación Gubernamental              11
Plaza                                   7
Pozo                                    6
Mercado                                 4
Tanque de Agua                          3
Cuerpo de Agua                          1
Name: GEOGRAFICO, dtype: int64

In [53]:
cruce_pe['TIPOMZA'].value_counts()

Típica              63649
Económica             242
Contenedora           154
Atípica                94
Contenida              75
Edificio-Manzana       71
Parque o Jardín        51
Name: TIPOMZA, dtype: int64

In [54]:
cruce_pe['TIPO'].value_counts()

Primaria             183
Camellón              69
Medio Superior        64
No  Aplica            53
No Aplica             45
Centro de Salud       20
Palacio Municipal     11
Jardín                 6
Agua                   6
Cancha                 6
Parque                 5
Preescolar             3
Unidad Deportiva       2
Tanque Elevado         2
Caja de Agua           1
Glorieta               1
Secundaria             1
Presa                  1
Name: TIPO, dtype: int64

In [49]:
# Ejes de calles
ejes_path = '/content/drive/MyDrive/Equipo_Agua/Geo/Data/DELEGACIONES_EDOMEX/TOLUCA_2018/LERMA/Ejes_Lerma.shp'
ejes = gpd.read_file(ejes_path)
ejes = ejes.to_crs(3857)
ejes.tail(2)

,STREETS,BANDA,geometry
7403,5 DE MAYO,342-E,"LINESTRING (-11077855.261 2188315.554, -110778..."
7404,20 DE NOVIEMBRE,341-E,"LINESTRING (-11077502.020 2188887.990, -110775..."


In [50]:
cruce_ejes = cruce_pe.sjoin(ejes)
print(cruce_ejes.shape)
'Porcentaje a borrar: ',cruce_ejes.shape[0]/cruce_pe.shape[0]*100

(10217, 25)


('Porcentaje a borrar: ', 7.123683091275458)

In [51]:
import numpy as np 
print('Shape original: ', cruce_pe.shape)
index_drop = np.unique(cruce_pe[cruce_pe.geometry.isin(cruce_ejes.geometry)].index.values)
cruce_pe.drop(index=index_drop,inplace=True)
print('Shape final:    ', cruce_pe.shape)

In [98]:
cruce_pe.columns

Index(['ID_casas', 'clase_dete', 'conf', 'area', 'Z', 'geometry', 'TIPOMZA',
       'TIPO', 'ID_15m', 'ID_15sip', 'ID_15sia', 'ID_15pe', 'CVE_AGEB',
       'CVE_MZA', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_SERV',
       'GEOGRAFICO', 'NOMSERV', 'AMBITO', 'CONDICION'],
      dtype='object')

In [52]:
cruce_pe.to_file(r'/content/drive/MyDrive/Equipo_Agua/Geo/Data/geoshaps/lerma/30mayo23/Lerma_casas_limpio.shp',
                 index=False)